In [ ]:
# класс CustomGradientBoostingRegressor


class CustomGradientBoostingRegressor(BaseEstimator):
    """
    Простой регрессор на основе градиентного бустинга над деревьями решений.

    Аргументы:
        n_estimators (int): Количество деревьев (итераций бустинга). По умолчанию — 100.
        learning_rate (float) Темп обучения (шаг градиентного спуска). По умолчанию — 0.1.
        max_depth (int): Максимальная глубина дерева бустинга. По умолчанию — 1.
        random_state : (int|None): Сид для фиксирования случайного состояния. По умолчанию — None (не фиксировать).

    Атрибуты:
        f0 (float): Начальное предсказание (среднее значение целевой переменной).
        models (list[DecisionTreeRegressor]): Последовательность деревьев, составляющих модель градиентного бустинга.
    """
    def __init__(self, n_estimators=100, learning_rate=0.1, max_depth=1, random_state=None):
        self.n_estimators = n_estimators
        self.learning_rate = learning_rate
        self.max_depth = max_depth
        self.random_state = random_state
        self.f0 = None
        self.models = []

    def fit(self, X, y):
        """
        Обучает модель градиентного бустинга.

        Аргументы:
            X (pandas.DataFrame): Таблица с признаками.
            y (numpy.ndarray): Массив значений целевой переменной.

        Возвращает: 
            CustomGradientBoostingRegressor: Обученная модель градиентного бустинга.
        """
        self.f0 = np.mean(y)
        y_pred = np.full_like(y, self.f0)
        for _ in range(self.n_estimators):
            residuals = y - y_pred
            tree = DecisionTreeRegressor(max_depth=self.max_depth, random_state=self.random_state)
            tree.fit(X, residuals)
            y_pred += self.learning_rate * tree.predict(X)
            self.models.append(tree)
        return self

    def predict(self, X):
        """
        Предсказывает значения целевой переменной.

        Аргументы:
            X (pandas.DataFrame): Таблица с признаками.

        Возвращает:
            numpy.ndarray: Массив предсказанных значений целевой переменной.
        """
        y_pred = np.full(X.shape[0], self.f0)
        for model in self.models:
            y_pred += self.learning_rate * model.predict(X)
        return y_pred
    
    def score(self, X, y):
        """
        Вычисляет отрицательное значение MSE (Negative MSE) для прогноза.
        Метод необходим для применения GridSearchCV.

        Аргументы:
            X (pandas.DataFrame): Таблица с признаками.
            y (numpy.ndarray): Массив значений целевой переменной.

        Возвращает:
            float: Значение Negative MSE.
        """
        return -mean_squared_error(y, self.predict(X))